<a href="https://colab.research.google.com/github/joost0252/DMML2022_Tudor/blob/main/Data_Mining_%26_Machine_Learning_Project_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Getting Started and all set up!


#### 1.1.Downloand and import any required packages.

In [1]:
# Download the french language model
!python -m spacy download fr_core_news_sm
# Download scikit-learn
!pip install scikit-learn
#Prerequisities for vectorization and other common libraries. 
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
#Prerequisities for the first four classification models. 
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
#Prerequisities for Neural Networks
import torch
import torch.nn as nn
from sklearn.decomposition import PCA
from torch.utils.data import Dataset, DataLoader

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-22 11:21:51.711510: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.3 MB 5.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Call the functions used
%run functions.py

In [3]:
#Setting our seeds so we have the same results every time we run. 
np.random.seed = 0
torch.manual_seed(0)

#### 1.2 Read training data and create as many unlabeled dataframes for our predictions as models we are training. 

In [4]:
#Corpus will be our training data for the whole set. 
corpus = pd.read_csv("training_data.csv")

In [5]:
unlabel_data_LR = pd.read_csv("unlabelled_test_data.csv")
unlabel_data_knn = pd.read_csv("unlabelled_test_data.csv")
unlabel_data_tree = pd.read_csv("unlabelled_test_data.csv")
unlabel_data_forest = pd.read_csv("unlabelled_test_data.csv")
unlabel_data_nn = pd.read_csv("unlabelled_test_data.csv")

In [6]:
submission_data = pd.read_csv('sample_submission.csv')

### 2. My models





#### 2.1.Baseline
What is the baseline for this classification problem?

In [7]:
base_rate = corpus['difficulty'].value_counts()/len(corpus['difficulty'])
base_rate.round(4)

A1    0.1694
C2    0.1681
C1    0.1662
B1    0.1656
A2    0.1656
B2    0.1650
Name: difficulty, dtype: float64

In [8]:
print(f"Base Rate:\n{base_rate.max()*100}%")

Base Rate:
16.9375%


#### 2.2. Logistic Regression (without data cleaning)

Train a simple logistic regression model using a Tfidf vectoriser.

In [9]:
tfidf_vector = TfidfVectorizer()

In [10]:
corpus = encode_difficulty(corpus)

In [11]:
#Spliting x and y, test and train.
X = corpus['sentence']
y= corpus['difficulty']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=0)

In [12]:
classifier = LogisticRegression()

In [13]:
pipe = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', classifier)])

# Fit model on training set
pipe.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', LogisticRegression())])

In [14]:
# Predictions
y_pred = pipe.predict(X_test)

Calculate accuracy, precision, recall and F1 score on the test set.


In [15]:
evaluate(y_test, y_pred)

CONFUSION MATRIX:
[[93 31 21 10  4  2]
 [54 60 30  6  6  8]
 [12 38 64 17  9 20]
 [ 6  6 15 66 27 24]
 [ 4  4 10 37 73 45]
 [ 7  8  8 19 24 92]]
ACCURACY SCORE:
0.4667
CLASSIFICATION REPORT:
	Precision: 0.4656
	Recall: 0.4667
	F1_Score: 0.4640


In [16]:
accuracyscore_lr = accuracy_score(y_pred, y_test)

Identify a few examples of sentences that are not well classified.

In [17]:
#Examples of sentences that are not well classified.
X_test[y_test != y_pred].head()

2255    C'est en décembre 1967, après bien des invecti...
608     Giscard va pourtant réussir à transformer ce r...
2856    Un choix difficile mais important : le public ...
1889    Le débat porte plutôt sur l'utilité d'une tell...
2358    Il faut du temps et du courage pour soigner to...
Name: sentence, dtype: object

Generate your first predictions on the `unlabelled_test_data.csv`. make sure your predictions match the format of the `unlabelled_test_data.csv`.

In [18]:
unlabel_data_LR['difficulty'] = pipe.predict(unlabel_data_LR['sentence'])

In [19]:
predictionsLR = prepare_submission(unlabel_data_LR)

In [20]:
probabilities_LR = get_prediction_probabilities(pipe, unlabel_data_LR)

Minimum value: 0.19
Maximum value: 0.82


In [21]:
predictionsLR.to_csv("Predictions_LR_tudor.csv", index=False)

#### 2.3. KNN (without data cleaning)

Train a KNN classification model using a Tfidf vectoriser. Show the accuracy, precision, recall and F1 score on the test set.

In [22]:
#We start with a value of p of default 2 and knn neighbours of default 5.
knn = KNeighborsClassifier(n_neighbors=5, weights = "uniform", p=2)

In [23]:
pipe_knn = Pipeline([('vectorizer', tfidf_vector),
                 ('KNN', knn)])

# Fit model on training set
pipe_knn.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('KNN', KNeighborsClassifier())])

In [24]:
# Predictions
y_pred_knn = pipe_knn.predict(X_test)

In [25]:
evaluate(y_test, y_pred_knn)

CONFUSION MATRIX:
[[121  28   8   1   1   2]
 [ 98  51  12   1   1   1]
 [ 81  39  33   3   1   3]
 [ 49  30  19  29   3  14]
 [ 48  36  29  15  29  16]
 [ 37  29  17  23   9  43]]
ACCURACY SCORE:
0.3187
CLASSIFICATION REPORT:
	Precision: 0.4030
	Recall: 0.3187
	F1_Score: 0.3022


Try to improve it by tuning the hyper parameters (`n_neighbors`,   `p`, `weights`).

In [26]:
# Define a grid of possible values for p
param_grid = {"KNN__p": [1, 2,]}

# Use GridSearchCV to search for the best value of p
grid = GridSearchCV(pipe_knn, param_grid, cv=5)
grid.fit(X_train, y_train)

# Print the best value of p found by the grid search
print("Best value of p: ", grid.best_params_["KNN__p"])

Best value of p:  2


In [27]:
# Define a grid of possible values for the weights
param_grid = {"KNN__weights": ['uniform', 'distance',]}

# Use GridSearchCV to search for the best weights
grid = GridSearchCV(pipe_knn, param_grid, cv=5)
grid.fit(X_train, y_train)

# Print the best weight parameter found by the grid search
print("Best weights: ", grid.best_params_["KNN__weights"])

Best weights:  distance


In [28]:
# Now that we know the best value for p and the best weight parameter let's find out the best value for the neighbours. 

for k in [1, 3, 5, 7, 9, 11, 13, 15]:
  knn = KNeighborsClassifier(n_neighbors=k, weights = "distance", p = 2)
  pipe_knn = Pipeline([('vectorizer', tfidf_vector),
                 ('KNN', knn)])

# Fit model on training set
  pipe_knn.fit(X_train, y_train)
  # Predictions
  y_pred_knn = pipe_knn.predict(X_test)
  print(f"ACCURACY SCORE(k={k}, p=2):\n{accuracy_score(y_test, y_pred_knn):.4f}")

ACCURACY SCORE(k=1, p=2):
0.3667
ACCURACY SCORE(k=3, p=2):
0.3615
ACCURACY SCORE(k=5, p=2):
0.3490
ACCURACY SCORE(k=7, p=2):
0.3594
ACCURACY SCORE(k=9, p=2):
0.3479
ACCURACY SCORE(k=11, p=2):
0.3542
ACCURACY SCORE(k=13, p=2):
0.3490
ACCURACY SCORE(k=15, p=2):
0.3521


In [29]:
#Now we have our best classifier:
knn2 = KNeighborsClassifier(n_neighbors=5, weights = "distance", p=2)
pipe_knn2 = Pipeline([('vectorizer', tfidf_vector),
                 ('KNN', knn2)])

# Fit model on training set
pipe_knn2.fit(X_train, y_train)
y_pred_knn2 = pipe_knn2.predict(X_test)
evaluate(y_test, y_pred_knn2)

CONFUSION MATRIX:
[[113  29  14   1   1   3]
 [ 87  56  15   4   1   1]
 [ 66  43  42   5   1   3]
 [ 33  29  21  36   3  22]
 [ 40  27  28  23  31  24]
 [ 34  24  14  17  12  57]]
ACCURACY SCORE:
0.3490
CLASSIFICATION REPORT:
	Precision: 0.4111
	Recall: 0.3490
	F1_Score: 0.3373


In [30]:
accuracyscore_knn = accuracy_score(y_pred_knn2, y_test)

In [31]:
#The accuracy has improved by 3 percentual points. 

Generate your first predictions on the `unlabelled_test_data.csv`. make sure your predictions match the format of the `unlabelled_test_data.csv`.

In [32]:
unlabel_data_knn['difficulty'] = pipe_knn2.predict(unlabel_data_knn['sentence'])

In [33]:
predictionsKNN = prepare_submission(unlabel_data_knn)

In [34]:
probabilities_KNN = get_prediction_probabilities(pipe, unlabel_data_knn)

Minimum value: 0.19
Maximum value: 0.82


#### 2.4. Decision Tree Classifier (without data cleaning)

Here we train a Decison Tree classifier, using a Tfidf vectoriser. We also show the accuracy, precision, recall and F1 score on the test set.

In [35]:
tree = DecisionTreeClassifier()

In [36]:
pipe_tree = Pipeline([('vectorizer', tfidf_vector),
                 ('Tree', tree)])

# Fit model on training set
pipe_tree.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('Tree', DecisionTreeClassifier())])

In [37]:
y_pred_tree = pipe_tree.predict(X_test)

In [38]:
evaluate(y_test, y_pred_tree)

CONFUSION MATRIX:
[[81 40 24  8  4  4]
 [46 55 35 17  4  7]
 [26 40 41 24 18 11]
 [ 7 17 29 42 28 21]
 [12 18 35 39 39 30]
 [13 13 30 34 34 34]]
ACCURACY SCORE:
0.3042
CLASSIFICATION REPORT:
	Precision: 0.3060
	Recall: 0.3042
	F1_Score: 0.3013


We  improve it by tuning the hyper parameters (`max_depth`, the depth of the decision tree).

In [39]:
# Define a grid of possible depths:
param_grid = {"Tree__max_depth": [1, 2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]}

# Use GridSearchCV to search for the best number of maximal depths:
grid = GridSearchCV(pipe_tree, param_grid, cv=5)
grid.fit(X_train, y_train)

# Print the best value of the max depth found by the grid search
print("Best value depth: ", grid.best_params_["Tree__max_depth"])

Best value depth:  19


In [40]:
#Now we have our best classifier:
tree2 = DecisionTreeClassifier(max_depth=15, random_state=0)
pipe_tree2 = Pipeline([('vectorizer', tfidf_vector),
                 ('Tree', tree2)])

# Fit model on training set
pipe_tree2.fit(X_train, y_train)
y_pred_tree2 = pipe_tree2.predict(X_test)
evaluate(y_test, y_pred_tree2)

CONFUSION MATRIX:
[[91 34 11 21  2  2]
 [68 51 19 17  2  7]
 [39 34 30 29 15 13]
 [16 12 19 66 14 17]
 [15 15 17 59 36 31]
 [22  9 21 48 24 34]]
ACCURACY SCORE:
0.3208
CLASSIFICATION REPORT:
	Precision: 0.3246
	Recall: 0.3208
	F1_Score: 0.3079


In [41]:
#The accuracy has improved by almost two percentual points.

In [42]:
accuracyscore_tree = accuracy_score(y_pred_tree2, y_test)

We have generated our  predictions on the `unlabelled_test_data.csv`. by matching the format of the `unlabelled_test_data.csv`.

In [43]:
unlabel_data_tree['difficulty'] = pipe_tree2.predict(unlabel_data_tree['sentence'])

In [44]:
predictionsTREE = prepare_submission(unlabel_data_tree)

In [45]:
probabilities_TREE = get_prediction_probabilities(pipe, unlabel_data_tree)

Minimum value: 0.19
Maximum value: 0.82


#### 2.5. Random Forest Classifier (without data cleaning)

We tried a Random Forest Classifier, using a Tfidf vectoriser. We showed the accuracy, precision, recall and F1 score on the test set.

In [46]:
forest = RandomForestClassifier()

In [47]:
pipe_forest = Pipeline([('vectorizer', tfidf_vector),
                 ('clf', forest)])

# Fit model on training set
pipe_forest.fit(X_train, y_train)
y_pred_forest = pipe_forest.predict(X_test)
evaluate(y_test, y_pred_forest)

CONFUSION MATRIX:
[[124  18  12   5   2   0]
 [ 70  63  21   8   1   1]
 [ 32  42  53  24   5   4]
 [ 17  13  16  66  20  12]
 [ 19  10  24  55  46  19]
 [ 16  13  10  33  23  63]]
ACCURACY SCORE:
0.4323
CLASSIFICATION REPORT:
	Precision: 0.4495
	Recall: 0.4323
	F1_Score: 0.4223


Tried to improve it by tuning the hyper parameters (`max_depth`, the depth of the decision tree).

In [48]:
# Define a grid 
param_grid = {"clf__max_depth": [None,1, 2,3,4,5,6,7,8,9,10],"clf__n_estimators": [10,50,100,200],"clf__min_samples_split": [2,5,10],"clf__min_samples_leaf": [1,2,5]}

# Use GridSearchCV to search for the best value of p
grid = GridSearchCV(pipe_forest, param_grid, cv=5)
grid.fit(X_train, y_train)
# Print the best value of the parameters found by the grid search
print("Best hyperparameters: ", grid.best_params_)

Best hyperparameters:  {'clf__max_depth': None, 'clf__min_samples_leaf': 2, 'clf__min_samples_split': 5, 'clf__n_estimators': 200}


In [49]:
#Now we have our best classifier:
forest2 = RandomForestClassifier(max_depth=None, n_estimators=200,min_samples_split=10,min_samples_leaf=1)
pipe_forest2 = Pipeline([('vectorizer', tfidf_vector),
                 ('clf', forest2)])

# Fit model on training set
pipe_forest2.fit(X_train, y_train)
y_pred_forest2 = pipe_forest2.predict(X_test)
evaluate(y_test, y_pred_forest2)

CONFUSION MATRIX:
[[117  20  13   9   1   1]
 [ 71  51  30   9   1   2]
 [ 34  33  53  25   6   9]
 [ 13   9  18  65  20  19]
 [ 16   6  19  55  48  29]
 [ 14  11  13  29  22  69]]
ACCURACY SCORE:
0.4198
CLASSIFICATION REPORT:
	Precision: 0.4286
	Recall: 0.4198
	F1_Score: 0.4101


In [50]:
accuracyscore_forest = accuracy_score(y_pred_forest2, y_test)

Here we generate our  predictions on the `unlabelled_test_data.csv`. make sure your predictions match the format of the `unlabelled_test_data.csv`.

In [51]:
unlabel_data_forest['difficulty'] = pipe_forest2.predict(unlabel_data_forest['sentence'])

In [52]:
predictionsFOREST = prepare_submission(unlabel_data_forest)

In [53]:
probabilities_FOREST= get_prediction_probabilities(pipe, unlabel_data_forest)

Minimum value: 0.19
Maximum value: 0.82


#### 2.6. Neural Networks including PCA


In [54]:
corpus_NN = pd.read_csv("training_data.csv")

In [55]:
# Use function for OrdinalEncoder
encode_difficulty(corpus_NN)

,id,sentence,difficulty
0,0,Les coûts kilométriques réels peuvent diverger...,4
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",0
2,2,Le test de niveau en français est sur le site ...,0
3,3,Est-ce que ton mari est aussi de Boston?,0
4,4,"Dans les écoles de commerce, dans les couloirs...",2
...,...,...,...
4795,4795,"C'est pourquoi, il décida de remplacer les hab...",3
4796,4796,Il avait une de ces pâleurs splendides qui don...,4
4797,4797,"Et le premier samedi de chaque mois, venez ren...",1
4798,4798,Les coûts liés à la journalisation n'étant pas...,5


In [56]:
#Define X as the sentences on which the prediction is based
X = corpus_NN['sentence']

#Define y as the difficulty that we want to predict
y = corpus_NN['difficulty']

In [57]:
#Use the TfidfVectorizer() on the X set
tfidf_vectorizer_vectors=tfidf_vector.fit_transform(X)

In [58]:
# Place tf-idf values in a pandas data frame
df = pd.DataFrame(tfidf_vectorizer_vectors.todense(), columns=tfidf_vector.get_feature_names())

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [59]:
# Define the PCA, which is used to reduce dimensions
pca = PCA(n_components=4800)

In [60]:
# Apply the PCA to the data frame
df = pca.fit_transform(df)

# Print the explained variance ratio of the PCA, the value being close to one, meaning that the PCA should be useful to use
print(sum(pca.explained_variance_ratio_))

1.0000000000000007


In [61]:
# Split the data in X_train, X_test, y_train, and y_test
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, shuffle=True, random_state=0)

In [62]:
# Transform y_test and y_train to numpy
y_test = y_test.to_numpy()
y_train= y_train.to_numpy()

In [63]:
# Reshape X_train and X_test to the right dimensions for the neural network
X_train = X_train.reshape(-1, X_train.shape[1]).astype('float32')
X_test = X_test.reshape(-1, X_test.shape[1]).astype('float32')

In [64]:
# Transform X_test and y_test to torch, such that it can be used by the neural network
X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)

In [65]:
# Define dataset class that can be used to load and process data for training the neural network
class Data(Dataset):
    def __init__(self):
        self.x=torch.from_numpy(X_train)
        self.y=torch.from_numpy(y_train)
        self.len=self.x.shape[0]
    def __getitem__(self,index):      
        return self.x[index], self.y[index]
    def __len__(self):
        return self.len

In [66]:
# Trainloader is used to create batched to train on
data_set = Data()
trainloader = DataLoader(dataset=data_set,batch_size=100)

In [67]:
# This class defines a neural network with two linear layers and applies a sigmoid activation function to the output of the first linear layer.
class Net(nn.Module):
    def __init__(self,D_in,H,D_out):
        super(Net,self).__init__()
        self.linear1=nn.Linear(D_in,H)
        self.linear2=nn.Linear(H,D_out)

        
    def forward(self,x):
        x=torch.sigmoid(self.linear1(x))  
        x=self.linear2(x)
        return x

In [68]:
input_dim=data_set.x.shape[1]     # Input based on the number of variables in the dataset
hidden_dim = 30                   # Number of nodes in hidden layer, based on testing
output_dim= 6                     #Output neural network based on number of classes

In [69]:
# Presets for the loop in the next cell
model = Net(input_dim,hidden_dim,output_dim)
criterion=nn.CrossEntropyLoss()
learning_rate=0.1
optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)

In [70]:
# Loop that checks that optimizes the model, aimed at minimizing the loss
n_epochs=1000
loss_list=[]

# Number of epochs
for epoch in range(n_epochs):
    for x, y in trainloader:
      

        # Clear the gradient 
        optimizer.zero_grad()
        # Make the prediction 
        z=model(x)
       
        loss=criterion(z,y)
        # Calculate gradients of the parameters 
        loss.backward()
        # Update the parameters 
        optimizer.step()
        
        loss_list.append(loss.data)
        

In [71]:
# Use the model for X_test
z = model(X_test)

In [72]:
# Put the values of z in a dataframe
px1 = pd.DataFrame(z.detach().numpy())

In [73]:
# Take the highest score in each row
max_col = px1.apply(lambda x: x.argmax(), axis=1)

In [74]:
# Transform the max_col to a numpy type
y_pred_nn_new = max_col.to_numpy()

In [75]:
accuracyscore_nn = accuracy_score(y_test, y_pred_nn_new)

In [76]:
# Print the confusion matrix, accuracy, precision, recall, and F1 score
evaluate(y_test, y_pred_nn_new)

CONFUSION MATRIX:
[[96 35 16  9  3  2]
 [51 69 30  7  4  3]
 [13 35 79 13  7 13]
 [ 6  7 16 79 18 18]
 [ 3  5 11 57 70 27]
 [ 4  5 10 32 30 77]]
ACCURACY SCORE:
0.4896
CLASSIFICATION REPORT:
	Precision: 0.4961
	Recall: 0.4896
	F1_Score: 0.4891


In [77]:
# Create tf-idf for the unlabeled data
tfidf_vectorizer_unlabeled = tfidf_vector.transform(unlabel_data_nn['sentence'])

# Create dataframe for the tf-idf
df_unlabel = pd.DataFrame(tfidf_vectorizer_unlabeled.todense(), columns=tfidf_vector.get_feature_names())

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [78]:
# Define PCA for the unlabelled data
pca_unlabel = PCA(n_components=1200)

# Use PCA on the unlabelled data
df_unlabel = pca.transform(df_unlabel)

In [79]:
# Reshape the df_unlabel such that it can be used within the neural network
df_unlabel = df_unlabel.reshape(-1, df_unlabel.shape[1]).astype('float32')

# Make a torch type of the df_unlabel
df_unlabel = torch.from_numpy(df_unlabel)

In [80]:
# Run the df_unlabel in the model
model_unlabel = model(df_unlabel)

In [81]:
# Transform to numpy type
px_unlabel = pd.DataFrame(model_unlabel.detach().numpy())

In [82]:
# Choose the highest row value and return the value of the column name
max_col_unlabel = px_unlabel.apply(lambda x: x.argmax(), axis=1)

# Transform to numpy type
pred_nn_unlabel = max_col_unlabel.to_numpy()

In [83]:
########### Now that we have the pred_nn_unlabel, we also should be able to generalize this part #####################

# Extract difficulty column from pred_nn_unlabel
unlabel_data_nn['difficulty'] = pred_nn_unlabel

In [84]:
# Transform values from integers to categorial values
unlabel_data_nn['difficulty'] = unlabel_data_nn['difficulty'].astype('category').cat.rename_categories(["A1", "A2", "B1", "B2", "C1", "C2"])

In [85]:
# Remove sentence column, such that is has the right format for Kaggle
predictionsnn = unlabel_data_nn.drop(columns=["sentence"])

In [86]:
# Create a csv file with the predicted values
predictionsnn.to_csv("Predictions_NN_tudor.csv", index=False)

### 3. Show a summary of your results

In [87]:
from tables.table import Table
d = {'Logistic Regression': [accuracyscore_lr], 'K-Nearest Neighbour': [accuracyscore_knn], 'Decision Tree Classifier': [accuracyscore_tree],'Random forest Classifier': [accuracyscore_forest],'Neural Networks': [accuracyscore_nn],}
index_labels=['Accuracy']
Table = pd.DataFrame(data=d, index=index_labels)
Table

,Logistic Regression,K-Nearest Neighbour,Decision Tree Classifier,Random forest Classifier,Neural Networks
Accuracy,0.466667,0.348958,0.320833,0.419792,0.489583
